In [1]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.agents.structured_output import ToolStrategy

from langgraph.checkpoint.memory import InMemorySaver 

from typing_extensions import TypedDict

from dotenv import load_dotenv
load_dotenv()

class EmailRequest(TypedDict):
    recipient: str
    subject: str
    body: str

USER = "Leo"


In [12]:
@tool()
def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient.
    Args:
        recipient: Email address of the recipient.
        subject: Subject line of the email.
        body: Body content of the email.
    Returns:
        Confirmation message.
    """
    return f"Email sent successfully to {recipient}"


agent = create_agent(
    model="openai:gpt-4o",
    tools=[send_email],
    system_prompt=f"You are a helpful assistant for {USER} that can send emails. Always return a message about the content of a tool call",
    middleware=[HumanInTheLoopMiddleware(interrupt_on={"send_email": True})],
    # response_format=ToolStrategy(EmailRequest),
    checkpointer=InMemorySaver(), # In production, use a persistent checkpointer like AsyncPostgresSaver.
)

In [32]:
# Choose decision
# Approval decision
approval = {
    "decisions": [
        {
            "type": "approve"
        }
    ]
}

# Edit decision
edit = {
    "decisions": [
        {
            "type": "edit",
            "edited_action": {
                "name": "send_email",
                "args": {
                    "recipient": "partner@startup.com",
                    "subject": "Budget proposal for Q1 2026",
                    "body": "I can only approve up to 500k, please send over details.",
                }
            }
        }
    ]
}

# Reject decision
reject = {
    "decisions": [
        {
            "type": "reject",
            "message": "Do not send an email. The email needs more context."
        }
    ]
}
# single dictionary to index into and return the correct dict
decisions = {
    "approve": approval,
    "edit": edit,
    "reject": reject,
}

# example usage:
# selected = decisions["edit"]
selected = decisions.get("edit")


In [33]:
import uuid
config = {"configurable": {"thread_id": str(uuid.uuid4())}}


In [ ]:

# user_input = "Send email to alice@example.com with subject Meeting and body Let's meet at 10 AM tomorrow."
# user_input += " The budget proposal for Q1 2026 is 1 million dollars."

user_input = "can you send an email to leo@gmail.com asking for a meeting tomorrow at 10am?"
result = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config
)

'''for msg in result["messages"]:

    msg.pretty_print()
    print("End of initial tool invocation output.\n\n")'''

from langgraph.types import Command
if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print("Human intervention required:", description)

    decision = edit  # Change this to approval or reject to test other flows

    result = agent.invoke(
        Command(
            resume=decisions.get("reject")
        ),
        config=config,
    )

for msg in result["messages"]:
    msg.pretty_print()

Human intervention required: Tool execution requires approval

Tool: send_email
Args: {'recipient': 'leo@breedt.com', 'subject': 'Meeting Request for Tomorrow', 'body': "Hi Leo,\n\nI hope this message finds you well. I would like to schedule a meeting with you tomorrow at 10am.\n\nPlease let me know if this time works for you or if there's a more convenient time.\n\nLooking forward to your confirmation.\n\nBest regards,\n[Your Name]"}
================================ Human Message =================================

can you send an email to leo@breedt.com asking for a meeting tomorrow at 10am?
================================== Ai Message ==================================
Tool Calls:
  send_email (call_jeS4jOd8buvAGgU8oUWIVHW8)
 Call ID: call_jeS4jOd8buvAGgU8oUWIVHW8
  Args:
    recipient: leo@breedt.com
    subject: Meeting Request for Tomorrow
    body: Hi Leo,

I hope this message finds you well. I would like to schedule a meeting with you tomorrow at 10am.

Please let me know if th

In [17]:
print(result["messages"][-1])

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 128, 'total_tokens': 209, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_65564d8ba5', 'id': 'chatcmpl-CZDvfFYlZLhBti0iD0noVrFGKb9Jz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--5e9abede-2492-490a-a445-b672f59c9bde-0' tool_calls=[{'name': 'send_email', 'args': {'recipient': 'leo@breedt.com', 'subject': 'Meeting Request for Tomorrow', 'body': 'Hello Leo,\n\nI hope this message finds you well. I would like to schedule a meeting for tomorrow at 10am. Please let me know if this time works for you.\n\nThank you.\n\nBest regards,\n[Your Name]'}, 'id': 'call_fbY3fCNtsghdyT1jJdSMEpzi',

In [22]:
print(result['__interrupt__'][-1].value['action_requests'][-1])

{'name': 'send_email', 'args': {'recipient': 'leo@breedt.com', 'subject': 'Meeting Request', 'body': 'Hi Leo,\n\nI hope this message finds you well. I would like to schedule a meeting with you tomorrow at 10 AM. Please let me know if that time works for you.\n\nBest regards,\n[Your Name]'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'recipient': 'leo@breedt.com', 'subject': 'Meeting Request', 'body': 'Hi Leo,\\n\\nI hope this message finds you well. I would like to schedule a meeting with you tomorrow at 10 AM. Please let me know if that time works for you.\\n\\nBest regards,\\n[Your Name]'}"}
